# Time Series Analytics with Pricing Data on Snowflake

This solution demonstrates several advanced time series features using FactSet Tick Data on Snowflake. You will learn to leverage powerful SQL functions such as TIME_SLICE, ASOF JOIN, and RANGE BETWEEN to gain deeper insights into time series trade data.

We're using `tick_history`, available through the Snowflake Marketplace, to gather trading information and have generated synthetic data for closing prices based on that. The `closing_prices` table was created as part of the setup and is available in the `raw` schema.

Please import below packages to get started:

- matplotlib=3.8.0
- seaborn=0.13.2

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

# Add a query tag to the session. This helps with debugging and performance monitoring.
session.query_tag = {"origin":"sf_sit", "name":"time_series_analysis", "version":{"major":1, "minor":0}}

#### Preview Data
We will be using FactSet Tick History data in this notebook. The data includes access high quality tick data sourced from FactSet’s real-time consolidated feed. In this notebook, we will focus on trade data from META. 


In [ ]:
SELECT TOP 100 * 
FROM tick_history.public.th_sf_mktplace
WHERE ticker='META' 
AND date =20221025
AND msg_type = 0 -- trade messages
AND security_type = 1; -- equity

We'll begin by formatting the data and filtering it for the META ticker, which we will use for future queries. The data from FactSet stores the date and time as integers, so we need to convert these into a proper timestamp format. This involves creating a trade_timestamp by extracting and reformatting the year, month, day, hour, minute, second, and nanoseconds from the integer fields. 

In [ ]:
SELECT 
    TIMESTAMP_FROM_PARTS(
        SUBSTR(date, 0, 4), -- year
        SUBSTR(date, 5, 2), -- month
        SUBSTR(date, 7, 2), -- day 
        SUBSTR(LPAD(time, 9, 0), 0, 2), -- hour
        SUBSTR(LPAD(time, 9, 0), 3, 2), -- minute
        SUBSTR(LPAD(time, 9, 0), 5, 2), -- second
        RPAD(SUBSTR(LPAD(time, 9, 0), 7, 3), 9, 0) -- nanoseconds
    ) AS trade_timestamp,
    ticker,
    last_price,
    last_vol
FROM tick_history.public.th_sf_mktplace
WHERE ticker = 'META'
AND msg_type=0
AND security_type = 1;

## Use Case 1: Prevailing Price

A common operation in FinServ is calculating the most recent price at which a security or asset was traded. This is a straightforward method and is often used in real-time trading environments. To do so for META, it can be done using simple SQL as follows: 

In [ ]:
SELECT *
FROM {{meta_trades}}
WHERE trade_timestamp <= '2022-10-10 12:00:00'
ORDER BY trade_timestamp DESC
LIMIT 1;

## Use Case 2: Trading Performance Tracking using TIME_SLICE
Investors and analysts often need to track the performance of a stock over a period of time for identifying trends such as increases or decreases in average price and trading volume before making purchase decisions. This can be easily done using the [TIME_SLICE](https://docs.snowflake.com/en/sql-reference/functions/time_slice) function in Snowflake. 

Note: The `TIME_SLICE` function in Snowflake segments timestamps into consistent intervals, like hours, days, or weeks, based on a chosen time unit. It returns the start or end of each interval, facilitating the aggregation of data within these defined time periods

#### Using TIME_SLICE
We will now use [TIME_SLICE](https://docs.snowflake.com/en/sql-reference/functions/time_slice) to get the average weekly trade price and total volume. Snowflake Notebooks allow you to [reference the results](https://docs.snowflake.com/en/user-guide/ui-snowsight/notebooks-develop-run#reference-cells-and-variables-in-sf-notebooks) of other cell queries using Jinja syntax.

In [ ]:
SELECT 
    TIME_SLICE(trade_timestamp, 1, 'WEEK', 'START') AS week_starting,
    AVG(last_price) AS average_price,
    SUM(last_vol) AS total_volume
FROM {{meta_trades}}
WHERE ticker='META'
GROUP BY week_starting
ORDER BY week_starting;

We can use Streamlit plots directly in our notebook to do a quick plot of average weekly price.

In [ ]:
st.line_chart(weekly_data, x="WEEK_STARTING", y="AVERAGE_PRICE")

The plot visualizes the average weekly closing price for the META ticker over time. Each point on the line chart represents the average price of META at the start of each week, allowing you to observe trends and fluctuations in the stock's performance.

Similarly, analysts can perform Long-Term Performance Tracking by using TIME_SLICE using the YEAR, QUARTER, MONTH, WEEK etc for identifying trends over longer periods.

#### Slice by Month

Let's find average montly price and total volume.

In [ ]:
SELECT 
    TIME_SLICE(trade_timestamp, 1, 'MONTH', 'START') AS month_starting,
    AVG(last_price) AS average_price,
    SUM(last_vol) AS total_volume
FROM {{meta_trades}}
WHERE ticker='META'
GROUP BY month_starting
ORDER BY month_starting;

#### Slice by Hour

Let's now slice by hour

In [ ]:
SELECT 
    TIME_SLICE(trade_timestamp, 1, 'HOUR', 'START') AS hour_starting,
    AVG(last_price) AS average_price,
    SUM(last_vol) AS total_volume
FROM {{meta_trades}}
WHERE DATE(trade_timestamp) = '2022-09-19'
AND ticker='META'
GROUP BY hour_starting
ORDER BY hour_starting;

## Use Case 3: Transaction Cost Analysis using ASOF JOIN

Analysts often want to determine how closely the trade prices align with the closing prices for trade quality analysis or get insight into the market impact of trades, showing whether trades are being executed at prices that significantly differ from the last known closing price, which might suggest market movements or anomalies.

We will use an [ASOF JOIN](https://docs.snowflake.com/en/sql-reference/constructs/asof-join) to join our trade data with closing price data, which we have stored in `closing_prices` table. The query calculates the price impact of each trade by comparing the trade price with the most recent closing price available at or before the time of the trade. 

In [ ]:
SELECT 
    TIMESTAMP_FROM_PARTS(
        SUBSTR(date, 0, 4), -- year
        SUBSTR(date, 5, 2), -- month
        SUBSTR(date, 7, 2), -- day 
        SUBSTR(LPAD(time, 9, 0), 0, 2), -- hour
        SUBSTR(LPAD(time, 9, 0), 3, 2), -- minute
        SUBSTR(LPAD(time, 9, 0), 5, 2), -- second
        RPAD(SUBSTR(LPAD(time, 9, 0), 7, 3), 9, 0) -- nanoseconds
    ) AS timestamp,
    ticker,
    closing_price
FROM raw.closing_prices
WHERE ticker = 'META';

In [ ]:
SELECT
    t1.ticker,
    t1.trade_timestamp,
    t1.last_price AS trade_price,
    t2.closing_price,
    trade_price - t2.closing_price AS price_impact,
    t1.last_vol
FROM 
     {{meta_trades}} t1
ASOF JOIN 
     {{meta_closing_prices}} t2
MATCH_CONDITION 
    (t1.trade_timestamp <= t2.timestamp)
ON 
    t1.ticker = t2.ticker
ORDER BY 
    t1.ticker,
    t1.trade_timestamp;


Now, let's do a daily summary analysis of trading data, focusing on key metrics like average trade price, average closing price, price impact, cumulative price impact, and total trading volume.

In [ ]:
SELECT 
    TIME_SLICE(trade_timestamp, 1, 'DAY', 'START') AS trade_date,
    AVG(trade_price) AS trade_price,
    AVG(closing_price) AS closing_price,
    AVG(price_impact) AS price_impact,
    SUM(price_impact) AS cumulative_price_impact,
    SUM(last_vol) AS total_volume
FROM {{transaction_cost}}
GROUP BY trade_date
ORDER BY trade_date;

We can also reference SQL cells in Python within the same notebook. Let's convert the daily sampled data to pandas for plotting. 

In [ ]:
df = transaction_cost_daily.to_pandas()

#### Trade Prices vs. Market Prices Over Time

In [ ]:
plt.figure(figsize=(14, 7))
sns.lineplot(x='TRADE_DATE', y='TRADE_PRICE', data=df, label='Trade Price', color='blue')
sns.lineplot(x='TRADE_DATE', y='CLOSING_PRICE', data=df, label='Market Price', color='red', linestyle='--')
plt.xlabel('Date')
plt.ylabel('Price')
plt.title('Trade Prices vs. Market Prices Over Time')
plt.legend()
plt.show()


This plot compares trade prices and market (closing) prices for a given dataset over time. The blue line represents the trade prices, while the red dashed line shows the market (closing) prices. This plot highlight the relationship and differences between the trade and closing prices across the selected time period.

#### Price Impact of Trades
Understanding the price impact helps in evaluating market efficiency. If trades consistently move prices significantly, it might indicate lower liquidity or higher volatility.
For traders, knowing the price impact helps in planning their trades to minimize market impact, especially for large orders. They might break up large orders or trade at specific times to reduce impact.

In [ ]:
plt.figure(figsize=(14, 7))
sns.scatterplot(x='TRADE_DATE', y='PRICE_IMPACT', data=df, alpha=0.5, color='purple')
plt.axhline(0, color='black', linestyle='--')
plt.xlabel('Date')
plt.ylabel('Price Impact')
plt.title('Price Impact of Trades Over Time')
plt.show()

This scatter plot shows the price impact of trades over time. Each point represents the price impact of a trade, with the plot helping to identify trends and patterns in how trades influenced market prices throughout the observed period. The horizontal reference line highlights whether the impact was positive or negative. Positive price impact might indicate bullish sentiment or aggressive buying. Negative price impact might indicate bearish sentiment or aggressive selling.

The distribution of META stock values shows varying densities, indicating that the price impact of trades fluctuates over time. High-density areas reflect periods when many trades clustered around similar price impacts, which could be due to high trading activity or specific market conditions affecting the stock. Low-density areas, conversely, suggest fewer trades or a wider range of price impacts. Analyzing these patterns can provide insights into periods of significant market influence or volatility, helping traders to understand how different trading conditions affect price impact.

#### Volume vs. Price Impact

In [ ]:
plt.figure(figsize=(14, 7))
sns.scatterplot(x='TOTAL_VOLUME', y='PRICE_IMPACT', data=df, alpha=0.5, color='green')
plt.xlabel('Volume')
plt.ylabel('Price Impact')
plt.title('Trade Volume vs. Price Impact')
plt.show()


This scatter plot depicts the relationship between trade volume and price impact. Each point represents a trade, with its position indicating how the trade volume correlates with the price impact. 

Most trades in the plot have volumes below 0.5 and price impacts ranging from -3 to 2. This distribution indicates that smaller trade volumes are common and their impact on price varies within a moderate range. It suggests that, for the data observed, the price impact remains relatively contained despite the low trade volumes, potentially reflecting a market where such trades have minimal influence on price movements.

#### Distribution of Price Impacts

In [ ]:
plt.figure(figsize=(14, 7))
sns.histplot(df['PRICE_IMPACT'], bins=50, color='orange', kde=True)
plt.xlabel('Price Impact')
plt.ylabel('Frequency')
plt.title('Distribution of Price Impacts')
plt.show()


This histogram displays the distribution of price impacts across trades. The plot uses 50 bins to show how frequently different price impact values occur, with the orange bars representing the frequency of each range and the KDE (Kernel Density Estimate) curve providing a smoothed representation of the distribution.

The histogram and KDE reveal a bell-shaped curve, indicating that price impacts are symmetrically distributed around a central value, suggesting a normal distribution.

#### Cumulative Price Impact

In [ ]:
st.line_chart(df.set_index('TRADE_DATE')['CUMULATIVE_PRICE_IMPACT'], use_container_width=True, color = ["#FF0000"])

This line chart shows how the cumulative price impact of trades changes over time. The red line indicates the trend in total price impact, revealing how it accumulates throughout the trading period.

### Slippage Calculation
As an alternative to using the closing price for our benchmark value, we can use the best bid offer ([BBO](https://databento.com/docs/examples/algo-trading/execution-slippage/overview)) price in addition to specifying a time window around our trade time. 

First, let’s specify a subset of trades we want to analyze. `mytrades` table has already been created as part of the set up.

In [ ]:
SELECT * 
FROM raw.mytrades
WHERE ticker = 'META'
AND TRADE_TIME BETWEEN '2022-10-25 9:30:00' AND '2022-10-25 16:00:00'

Second, we can calculate the difference between our trade price vs the BBO price that is closest to our trade time, as well as the difference of the BBO price 1000 ms later.

In [ ]:
WITH q AS (
    SELECT
        TIMESTAMP_FROM_PARTS(
            SUBSTR(date, 0, 4),            -- year
            SUBSTR(date, 5, 2),            -- month
            SUBSTR(date, 7, 2),            -- day
            SUBSTR(LPAD(time, 9, 0), 0, 2), -- hour
            SUBSTR(LPAD(time, 9, 0), 3, 2), -- minute
            SUBSTR(LPAD(time, 9, 0), 5, 2), -- second
            RPAD(SUBSTR(LPAD(time, 9, 0), 7, 3), 9, 0) -- nanoseconds
        ) AS trade_timestamp,
        *
    FROM tick_history.public.th_sf_mktplace 
    WHERE ticker = 'META'
    AND trade_timestamp BETWEEN '2022-10-25 09:30:00' AND '2022-10-25 16:30:00'
    AND msg_type = 15
)
SELECT
    (q.ask + q.bid) / 2 AS mid_price_trade,
    (qp.ask + qp.bid) / 2 AS mid_price_markout,
    price AS trade_price,
    shares AS trade_size
FROM
    {{mytrades}} t
    ASOF JOIN q 
    MATCH_CONDITION (t.trade_time >= q.trade_timestamp) 
    ON t.ticker = q.ticker -- same cardinality
    ASOF JOIN q qp 
    MATCH_CONDITION (TIMESTAMPADD(ms, 1000, t.trade_time) >= qp.trade_timestamp) 
    ON t.ticker = qp.ticker;


Finally we can calculate the markout value, showing us the impact from our trade.

In [ ]:
SELECT
    0::INT AS markout_delay_ms,
    1e4*SUM(ABS(mid_price_markout-mid_price_trade)*trade_size)/SUM(trade_size*trade_price) AS abs_markout --1e4 basis points
FROM
    {{bbo_price}}

## Use Case 4: Trend Analysis using RANGE-based sliding window

Intraday analysis is valuable for analysts, offering real-time insights into the short-term price movements of a stock. It helps traders and analysts monitor price trends within a time window, aiding in decision-making and strategy development throughout the trading day. This can be easily done using the RANGE-based sliding windows in Snowflake. 

Note: A range-based window frame consists of a logically computed set of rows rather than a physical number of rows as would be expressed in a row-based frame. Let's explore Range Between to create interesting time series metrics on our data.

Instead of calculating the average for every trade timestamp, aggregate the data by regular intervals and then compute the moving average. This approach reduces the granularity of your data.

In [ ]:
SELECT
    DATE_TRUNC('minute', trade_timestamp) AS interval_start,
    ticker,
    AVG(last_price) AS avg_price,
    AVG(last_vol) AS avg_vol
FROM 
    {{meta_trades}}
WHERE 
    DATE(trade_timestamp) = '2022-06-09'
GROUP BY
    DATE_TRUNC('minute', trade_timestamp),
    ticker

This query calculates the 10-minute moving average of the last_price for META trades throughout the trading day on June 9, 2022. 
A range-based [window frame](https://docs.snowflake.com/en/sql-reference/functions-analytic) consists of a logically computed set of rows rather than a physical number of rows as would be expressed in a row-based frame. Let's explore Range Between to create interesting time series metrics on our data.

In [ ]:
SELECT
    interval_start AS trade_timestamp,
    ticker,
    AVG(avg_price) OVER (
        PARTITION BY ticker
        ORDER BY interval_start 
        RANGE BETWEEN INTERVAL '10 MINUTE' PRECEDING AND CURRENT ROW
    ) AS moving_avg
FROM 
    {{aggregated_data}};

This query calculates the volume-weighted average price  (VWAP) for each ticker over a rolling 10-minute window throughout the trading day on June 9, 2022. VWAP is a key metric used by traders and analysts to understand the average price at which a stock has traded, weighted by the volume of trades. It provides a more accurate reflection of the stock’s price level considering the size of each trade

In [ ]:
SELECT 
    ticker,
    interval_start AS trade_timestamp,
    avg_price,
    avg_vol,
    SUM(avg_price * avg_vol) OVER (
        PARTITION BY ticker 
        ORDER BY trade_timestamp 
        RANGE BETWEEN INTERVAL '10 MINUTE' PRECEDING AND CURRENT ROW
    ) / 
    SUM(avg_vol) OVER (
        PARTITION BY ticker 
        ORDER BY trade_timestamp 
        RANGE BETWEEN INTERVAL '10 MINUTE' PRECEDING AND CURRENT ROW
    ) AS volume_weighted_avg
FROM {{aggregated_data}};

This query calculates the time-weighted average price (TWAP) for each ticker over a rolling 10-minute window throughout the trading day on June 9, 2022. TWAP measures the average price of a stock weighted by the duration each price level was active. It provides a more accurate representation of the stock’s price level by considering the time each price was held, helping traders and analysts understand price trends over time.

In [ ]:
WITH trade_durations AS (
    SELECT
        ticker,
        interval_start AS trade_timestamp,
        avg_price,
        avg_vol,
        LAG(trade_timestamp) OVER (
            PARTITION BY ticker 
            ORDER BY trade_timestamp
        ) AS prev_trade_timestamp
    FROM {{aggregated_data}}
),
price_weighted AS (
    SELECT
        ticker,
        trade_timestamp,
        avg_price,
        avg_vol,
        CASE
            WHEN prev_trade_timestamp IS NULL THEN 0
            ELSE DATEDIFF(SECOND, prev_trade_timestamp, trade_timestamp)
        END AS duration_seconds
    FROM trade_durations
),
time_weighted_avg AS (
    SELECT
        ticker,
        trade_timestamp,
        CASE
            WHEN SUM(duration_seconds) OVER (
                PARTITION BY ticker 
                ORDER BY trade_timestamp 
                RANGE BETWEEN INTERVAL '10 MINUTE' PRECEDING AND CURRENT ROW
            ) = 0 THEN 0
            ELSE SUM(avg_price * duration_seconds) OVER (
                PARTITION BY ticker 
                ORDER BY trade_timestamp 
                RANGE BETWEEN INTERVAL '10 MINUTE' PRECEDING AND CURRENT ROW
            ) / 
            SUM(duration_seconds) OVER (
                PARTITION BY ticker 
                ORDER BY trade_timestamp 
                RANGE BETWEEN INTERVAL '10 MINUTE' PRECEDING AND CURRENT ROW
            )
        END AS time_weighted_avg
    FROM price_weighted
)
SELECT
    ticker,
    trade_timestamp,
    time_weighted_avg
FROM time_weighted_avg
WHERE time_weighted_avg != 0;

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

df_moving_avg = moving_avg.to_pandas()
df_vwap = volume_weighted.to_pandas()
df_time_weighted_avg = time_weighted_avg.to_pandas()
df_combined = pd.merge(df_moving_avg, df_vwap, on=['TRADE_TIMESTAMP', 'TICKER'], how='outer')
df_combined = pd.merge(df_combined, df_time_weighted_avg, on=['TRADE_TIMESTAMP', 'TICKER'], how='outer')

# Ensure no duplicate columns
df_combined = df_combined[['TRADE_TIMESTAMP', 'MOVING_AVG', 'VOLUME_WEIGHTED_AVG', 'TIME_WEIGHTED_AVG']]
df_combined

In [ ]:
plt.figure(figsize=(14, 7))

# Plot Moving Average
sns.lineplot(x='TRADE_TIMESTAMP', y='MOVING_AVG', data=df_combined, label='Moving Average', color='green')

# Plot VWAP
sns.lineplot(x='TRADE_TIMESTAMP', y='VOLUME_WEIGHTED_AVG', data=df_combined, label='VOLUME_WEIGHTED_AVG', color='blue')

# Plot TWAP
sns.lineplot(x='TRADE_TIMESTAMP', y='TIME_WEIGHTED_AVG', data=df_combined, label='TIME_WEIGHTED_AVG', color='red')

plt.xlabel('Date')
plt.ylabel('Price')
plt.title('Moving Average, VWAP, and TWAP Over Time')
plt.legend()
plt.show()

As the plot shows, the moving averages, VWAP, and TWAP are behaving consistently, with no significant volatility or divergence in price trends, indicating overall stability for this stock.

## Use Case 5: Volatility assessment using LEAD & LAG

The analysis of price changes between consecutive trades, providing insights into how the price of a stock is evolving throughout the trading day. This query assesses the price of the previous & next trade wrt the current trade using [LEAD](https://docs.snowflake.com/en/sql-reference/functions/lead) & [LAG](https://docs.snowflake.com/en/sql-reference/functions/lag) window functions.  

We'll start by getting the previous trade price using `LAG`.

In [ ]:
SELECT 
    ticker,
    trade_timestamp,
    last_price,
    LAG(last_price, 1) OVER (
        PARTITION BY ticker 
        ORDER BY trade_timestamp
    ) AS previous_price
FROM {{meta_trades}}
WHERE DATE(trade_timestamp) = '2022-06-09'

We'll now use `LEAD` to get the next immediate trade. 

In [ ]:
SELECT 
    ticker,
    trade_timestamp,
    last_price,
    LEAD(last_price, 1) OVER (
        PARTITION BY ticker 
        ORDER BY trade_timestamp
    ) AS next_price
FROM {{meta_trades}}
WHERE DATE(trade_timestamp) = '2022-06-09'